In [1]:
# preprocess_data.py (versión simplificada para manual introductorio)

import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess_data(input_path='predict_income_2020.csv', output_path='predict_income_2020_2.csv'):
    """
    Carga los datos crudos, los limpia, preprocesa y guarda un archivo CSV listo para el modelado.
    
    SIMPLIFICACIÓN METODOLÓGICA:
    Este script filtra y se queda ÚNICAMENTE con las observaciones donde el ingreso es mayor a cero.
    Esta decisión se toma para simplificar el problema en un contexto introductorio.

    Pasos de preprocesamiento:
    1. Carga los datos.
    2. FILTRA los datos para mantener solo ingmo2hd > 0.
    3. Elimina columnas constantes.
    4. Transforma 'ingmo2hd' a una escala logarítmica (ahora sin ceros, np.log es suficiente).
    5. Aplica StandardScaler a numéricas y OneHotEncoder a categóricas.
    6. Guarda el DataFrame procesado en un nuevo archivo CSV.
    """
    print("--- Iniciando el script de preprocesamiento (versión simplificada) ---")

    # --- PASO 1: Carga y limpieza inicial ---
    data = pd.read_csv(input_path, encoding="latin-1")
    if 'Unnamed: 0' in data.columns:
        data = data.drop('Unnamed: 0', axis=1)
    
    # --- PASO 2: FILTRADO DE DATOS (EL PASO CLAVE) ---
    original_rows = len(data)
    data_positive_income = data[data['ingmo2hd'] > 0].copy()
    filtered_rows = len(data_positive_income)
    print(f"Filtrado de datos: Se eliminaron {original_rows - filtered_rows} observaciones con ingreso cero.")
    print(f"El dataset ahora contiene {filtered_rows} observaciones.")
    
    # --- PASO 3: Eliminar columnas constantes (después del filtrado) ---
    cols_to_drop = [col for col in data_positive_income.columns if data_positive_income[col].nunique() == 1]
    data_cleaned = data_positive_income.drop(columns=cols_to_drop)
    print(f"Se eliminaron {len(cols_to_drop)} columnas constantes.")
    
    # --- PASO 4: Transformar la variable objetivo ---
    # Como ya no hay ceros, podemos usar np.log. np.log1p sigue siendo seguro y una buena práctica.
    data_cleaned['log_ingmo2hd'] = np.log1p(data_cleaned['ingmo2hd'])
    data_cleaned = data_cleaned.drop(columns=['ingmo2hd'])
    
    # Separar características (X) y objetivo (y)
    X = data_cleaned.drop('log_ingmo2hd', axis=1)
    y = data_cleaned['log_ingmo2hd']

    # --- PASO 5: Preprocesamiento de X ---
    numeric_features = ['p208a_20', 'p104_20', 'p301b_20']
    categorical_features = [col for col in X.columns if col not in numeric_features]

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', drop='if_binary', sparse_output=False), categorical_features)
        ],
        remainder='passthrough'
    )
    
    X_processed_array = preprocessor.fit_transform(X)
    new_feature_names = preprocessor.get_feature_names_out()
    X_processed_df = pd.DataFrame(X_processed_array, columns=new_feature_names)
    
    # --- PASO 6: Combinar y Guardar ---
    final_df = pd.concat([X_processed_df, y.reset_index(drop=True)], axis=1)

    final_df.to_csv(output_path, index=False)
    print(f"\n--- ¡Proceso completado! ---\nDatos listos para modelar (solo ingresos positivos) guardados en: '{output_path}'")


if __name__ == "__main__":
    preprocess_data()

--- Iniciando el script de preprocesamiento (versión simplificada) ---
Filtrado de datos: Se eliminaron 1748 observaciones con ingreso cero.
El dataset ahora contiene 6193 observaciones.
Se eliminaron 14 columnas constantes.

--- ¡Proceso completado! ---
Datos listos para modelar (solo ingresos positivos) guardados en: 'predict_income_2020_2.csv'
